In [32]:
from sklearn import datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from fitter import Fitter
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn import utils
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer 
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer
import missingno as msno
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import roc_auc_score

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [3]:
 #Carga de los archivos
 
df= pd.read_csv(r'C:\Users\Lonelywolf\Documents\PI_02\Datathon\properties_colombia_train.csv')
df_test=pd.read_csv(r'C:\Users\Lonelywolf\Documents\PI_02\Datathon\properties_colombia_test.csv')

In [4]:
#Eliminamos las columnas que no vamos a utilizar

df.drop(columns= ['id', 'l1', 'l5', 'l6', 'surface_covered','surface_total','l4','operation_type','description', 'title', 'price_period','created_on','end_date','ad_type','start_date','geometry'], inplace=(True))
df_test.drop(columns= ['id', 'l1', 'l5', 'l6', 'surface_covered','surface_total','l4','operation_type','description', 'title', 'price_period','created_on','end_date','ad_type','start_date','geometry'], inplace=(True))

In [5]:
df.drop(columns= ['l2', 'l3',], inplace=(True))
df_test.drop(columns= ['l2', 'l3'], inplace=(True))

In [6]:
# Rellenamos los valores nulos con el promedio de los valores de la tabla precio

df['price'].fillna(643605091, inplace=True)

In [7]:
# Reemplazamos el valor "-2147483648" repetido varias veces con un valor con valores con la misma cantida de digitos

df['price'].replace(-2147483648, np.random.uniform(2000000000, 400000000), inplace=True)
df['price']= df['price'].astype(int)

In [8]:
# Reemplazamos el tipo de moneda

df['currency'].replace('USD', 'COP', inplace=True)
df_test['currency'].replace('USD', 'COP', inplace=True)

In [9]:
# Rellenamos los valores nulos de las columnas con 0

df['rooms'].fillna(0, inplace=True)
df['bedrooms'].fillna(0, inplace=True)
df['bathrooms'].fillna(0, inplace=True)

df_test['bedrooms'].fillna(0, inplace=True)
df_test['rooms'].fillna(0, inplace=True)
df_test['bathrooms'].fillna(0, inplace=True)

In [10]:
# Sumamos los valores de las columnas "bedrooms" y "bathrooms" a la columa "rooms" y le sumamos su promedio para tener una columna unificada

df['rooms']= df['rooms'] + df['bedrooms']+ df['bathrooms']
df['rooms']= df['rooms'] + df['rooms'].mean()
df['rooms']= df['rooms'].astype(int)

df_test['rooms']= df_test['rooms'] + df_test['bedrooms']+ df_test['bathrooms']
df_test['rooms']= df_test['rooms'] + df_test['rooms'].mean()
df_test['rooms']= df_test['rooms'].astype(int)

In [11]:
# Eliminamos las columnas "bathrooms", "bedrooms"

df.drop(columns=['bathrooms', 'bedrooms'], inplace=True)
df_test.drop(columns=['bathrooms', 'bedrooms'], inplace=True)

In [13]:
# Eliminamos valores nulos

df.dropna(inplace=True)
df_test.dropna(inplace=True)

In [14]:
promedio= df['price'].mean()

In [15]:
df['target']= df['price'] < promedio

In [ ]:
df.drop(columns='price', inplace=True)

In [16]:
# Creamos con dataframe con variables dummies para obtener valores binarios

df= pd.get_dummies(df,columns=['currency','property_type','target'], drop_first=True)
df_test=pd.get_dummies(df_test,columns=['currency','property_type'], drop_first=True)

In [333]:
# Analizamos las distribuciones

distribuciones = ['cauchy', 'chi2', 'expon',  'exponpow', 'gamma',
                  'norm', 'powerlaw', 'beta', 'logistic']

fitter = Fitter(df['price'], distributions=distribuciones)
fitter.fit()
fitter.summary(Nbest=10, plot=False)

Fitting 9 distributions: 100%|██████████| 9/9 [00:11<00:00,  1.33s/it]


,sumsquare_error,aic,bic,kl_div,ks_statistic,ks_pvalue
cauchy,1.316123e-17,4582.330958,-6.960508e+06,0.186672,0.173151,0.0
gamma,1.337498e-17,4464.916901,-6.958285e+06,0.228057,0.100204,0.0
beta,1.346759e-17,4464.571581,-6.957326e+06,0.230575,0.100763,0.0
expon,2.227596e-17,4435.162702,-6.888261e+06,0.401237,0.146035,0.0
logistic,2.351965e-17,4558.403514,-6.880803e+06,0.342208,0.159329,0.0
exponpow,2.353606e-17,4431.964436,-6.880695e+06,0.441804,0.145355,0.0
norm,2.925190e-17,4515.776073,-6.850858e+06,0.485374,0.192496,0.0
powerlaw,3.791957e-17,4331.761706,-6.815216e+06,0.912742,0.252567,0.0
chi2,6.367745e-17,inf,-6.744050e+06,NaN,0.999985,0.0


In [59]:
y= df['target_True'].values

X=df.drop('target_True', axis=1).values

In [60]:
#Realizamos el entrenamiento y la prediccion del archivo "properties_colombia_train.csv"

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

knn= KNeighborsClassifier(n_neighbors=6)
knn.fit(X_train, y_train)
y_predict= knn.predict(X_test)
knn.score(X_test, y_test)



0.7506024910470956

In [65]:
# Realizamos una validacion cruzada 
cv_scores= cross_val_score(knn, X, y, cv=5, scoring='roc_auc')
print(cv_scores.mean())

0.8124534794818217


In [57]:
# Realizamos la prediccion con el archivo "properties_colombia_test.csv"
X= df_test.values
y_predict= knn.predict(X)

In [66]:
confusion_matrix(y_test, y_predict)

array([[10334,  5030],
       [ 6043, 22992]], dtype=int64)

In [67]:
classification_report(y_test, y_predict)

'              precision    recall  f1-score   support\n\n           0       0.63      0.67      0.65     15364\n           1       0.82      0.79      0.81     29035\n\n    accuracy                           0.75     44399\n   macro avg       0.73      0.73      0.73     44399\nweighted avg       0.75      0.75      0.75     44399\n'

In [ ]:
# Guardamos el resutaldo de la prediccion

guardado= pd.DataFrame({'pred':y_predict})
guardado.to_csv('Lonelywolf2701.csv', index=False)
